# ニューラルネットワークの訓練

## 目的関数(損失関数)
分類問題によく使われる交差エントロピー誤差について触れる

In [1]:
import numpy as np

def cross_entropy_error(y, t):
    print("cross_entropy_error : ")
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

T = np.array([0, 1, 0, 0, 0])

y1 = np.array([0.1, 0.6, 0.2, 0.0, 0.0])
y2 = np.array([0.1, 0.2, 0.05, 0.0, 0.0])

print(cross_entropy_error(y1, T))
print("=" * 40)
print(cross_entropy_error(y2, T))

cross_entropy_error : 
0.510825457099338
cross_entropy_error : 
1.6094374124342252


正解となるラベルの出力が小さいと交差エントロピー誤差が大きくなることがわかる

## 目的関数の最小化
目的関数(損失関数)の値を最小にするようなパラメータの値を求めることで、ニューラルネットワークを訓練する.

ここでは勾配降下法に触れる.

In [3]:
"""
numpyを使って勾配の計算をしてみる.
バイアスは全て初期化されているとする.
"""
# 入力
x = np.array([2, 3, 1])

# 正解
t = np.array([20])

In [4]:
# パラメータの定義

# 1層目と 2層目の間の線形変換のパラメータ
# 3次元ベクトルを2次元ベクトルに変換
w1 = np.array([[3, 1, 2], [-2, -3, -1]])
# 二次元バイアス
b1 = np.array([0, 0])

# 2層目と 3層目の間の線形変換のパラメータ
# 2次元ベクトルを1次元ベクトルに変換
w2 = np.array([[3, 2]])
# 一次元バイアス
b2 = np.array([0])

In [5]:
# 中間層の計算
# 活性化関数にReLuを用いる
u1 = w1.dot(x) + b1
h1 = np.maximum(0, u1)

# 出力の計算
y = w2.dot(h1) + b2

print(y)

[33]


In [6]:
# dL/dw2 = dL/dy * dy/dw2 を求めたい

# dL / dy
dLdy = -2 * (t - y)

# dy / dw_2
dydw2 = h1

In [7]:
# dL / dw_2: 求めたい勾配
dLdw2 = dLdy * dydw2

print(dLdw2)

[286   0]


In [8]:
# d y / d h1
dydh1 = w2

# d h1 / d u1
dh1du1 = np.where( h1 > 0, 1, 0)

# d u_1 / d w1
du1dw1 = x

# 上から du1 / dw1 の直前までを一旦計算
dLdu1 = dLdy * dydh1 * dh1du1

# du1dw1は (3,) というshapeなので、g_u1w1[None]として(1, 3)に変形
du1dw1 = du1dw1[None]

# dL / dw_1: 求めたい勾配
dLdw1 = dLdu1.T.dot(du1dw1)

print(dLdw1)

[[156 234  78]
 [  0   0   0]]


ここまでをchainerで実装すると以下のようになる.

In [9]:
import chainer
import chainer.functions as F
import chainer.links as L

# 1-2層間のパラメータ
w1 = np.array([[3, 1, 2], [-2, -3, -1]], dtype=np.float32)
b1 = np.array([0, 0], dtype=np.float32)

l1 = L.Linear(2, initialW=w1, initial_bias=b1)

# 2-3層間のパラメータ
w2 = np.array([[3, 2]], dtype=np.float32)
b2 = np.array([0], dtype=np.float32)

l2 = L.Linear(1, initialW=w2, initial_bias=b2)

# 入力
x = np.array([[2, 3, 1]], dtype=np.float32)

# 出力
y = l2(F.relu(l1(x)))
print(y)  # => should be [2.99995156]

# 正解
t = np.array([[20.]], dtype=np.float32)

# ロス
loss = F.mean_squared_error(y, t)

# dLdw_2
print(chainer.grad([loss], [l2.W]))  # => should be [-3.39995290e+01 -2.82720335e-05]

# dLdw_1
print(chainer.grad([loss], [l1.W]))  # => should be [[-3.40704286e-03 -5.11056429e-03 -1.70352143e-03]
                                                  #                      [-1.13088040e-04 -1.69632060e-04 -5.65440200e-05]]

/Users/yosuke/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


variable([[33.]])
[variable([[286.,   0.]])]
[variable([[156., 234.,  78.],
          [  0.,   0.,   0.]])]
